In [ ]:
# If in Colab, then import the drive module from google.colab
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  # Mount the Google Drive to access files stored there
  drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# standard data science librraies for data handling and visualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import re
import gc
import time

In [ ]:
df_train = pd.read_csv('/content/drive/My Drive/utd_graduation_data.csv')

In [ ]:
df_train.head(10)

,URL,MainTitle,SectionTitle,Content
0,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,Final Exam Schedule,Find your final exam schedule in your Student ...
1,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,The Summer and Fall Course Schedule is Now Ava...,Your date and time to register is your enrollm...
2,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,Need to Waitlist for a Class?,NaN
3,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,Mobile Registration,Registration is now easier using Schedule Plan...
4,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,Don’t Wait to Validate,Use Schedule Planner to validate your schedule...
5,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,Check Your UTD Email Often,The Registrar’s Office communicates important ...
6,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,Review UT Dallas Course Catalogs Often,The academic catalogs will provide you relevan...
7,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,Orion: Your Student Account,Register for courses Obtain your enrollment ve...
8,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,Important Dates,NaN
9,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,NaN,NaN


In [ ]:
pip install ftfy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.1 MB/s eta 0:00:00


In [ ]:
pip install chardet


In [ ]:
import chardet

# Open the file in binary mode to detect encoding
with open('/content/drive/My Drive/utd_graduation_data.csv', 'rb') as f:
    raw_data = f.read()
    result = chardet.detect(raw_data)
    print(result)


{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [ ]:
import pandas as pd
import re
from ftfy import fix_text
import unicodedata

# --------- Load the CSV ---------
try:
    df_train = pd.read_csv('/content/drive/My Drive/utd_graduation_data.csv', encoding='utf-8')
except UnicodeDecodeError:
    df_train = pd.read_csv('/content/drive/My Drive/utd_graduation_data.csv', encoding='latin1') #if utf-8 fails, try latin1

# --------- Optional: Define extra manual mappings (if needed) ---------
manual_map = {
    'â€“': '–',
    'â€™': '’',
    'â€œ': '“',
    'â€': '”',
    'â€˜': '‘',
    'â€¢': '•'
    # Add more as you discover them
}

def apply_manual_map(text):
    if pd.isnull(text):
        return text
    for bad, good in manual_map.items():
        text = str(text).replace(bad, good)
    return text

# --------- Clean function using ftfy + manual map + robust regex ---------
def clean_text(text):
    if pd.isnull(text):
        return text
    text = fix_text(str(text))  # Smart fix
    text = apply_manual_map(text)  # Manual replacements

    def is_valid_char(char):
        if ord(char) < 128:  # ASCII
            return True
        elif char in '–’“”•€':  # Explicitly allowed
            return True
        elif unicodedata.category(char)[0] == 'L': #Allow letters
            return True
        elif unicodedata.category(char)[0] == 'N': #Allow Numbers
            return True
        elif unicodedata.category(char)[0] == 'P': #Allow Punctuation
            return True
        elif unicodedata.category(char)[0] == 'S': #Allow Symbols
            return True
        elif char == ' ': #Allow spaces
            return True
        else:
            return False

    text = ''.join(char if is_valid_char(char) else ' ' for char in text) # Replace bad characters with spaces.
    return text

# --------- Choose columns to clean ---------
columns_to_clean = ['MainTitle', 'SectionTitle', 'Content']

for col in columns_to_clean:
    if col in df_train.columns:
        df_train[col] = df_train[col].apply(clean_text)

# --------- Drop fully null columns ---------
df = df_train.dropna(axis=1, how='all')

# --------- Save the cleaned output ---------
df.to_csv("utd_cleaned_output.csv", index=False)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

print("✅ Done! Cleaned file saved as 'utd_cleaned_output.csv'")

output_path = '/content/drive/MyDrive/utd_cleaned_output.csv'
df.to_csv(output_path, index=False, encoding ='utf-8')
print(f"✅ Saved to: {output_path}")

# output_path = '/content/drive/MyDrive/cleaned_output_utf8.csv'
# df.to_csv(output_path, index=False, encoding='utf-8')
# print(f"✅ Done! Saved cleaned file to: {output_path}")

✅ Done! Cleaned file saved as 'utd_cleaned_output.csv'
✅ Saved to: /content/drive/MyDrive/utd_cleaned_output.csv


In [ ]:
df.head(20)


,URL,MainTitle,SectionTitle,Content
0,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,Final Exam Schedule,Find your final exam schedule in your Student ...
1,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,The Summer and Fall Course Schedule is Now Ava...,Your date and time to register is your enrollm...
2,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,Need to Waitlist for a Class?,NaN
3,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,Mobile Registration,Registration is now easier using Schedule Plan...
4,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,Don't Wait to Validate,Use Schedule Planner to validate your schedule...
5,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,Check Your UTD Email Often,The Registrar's Office communicates important ...
6,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,Review UT Dallas Course Catalogs Often,The academic catalogs will provide you relevan...
7,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,Orion: Your Student Account,Register for courses Obtain your enrollment ve...
8,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,Important Dates,NaN
9,https://registrar.utdallas.edu/,Office of the Registrar | The University of Te...,NaN,NaN


In [ ]:
def inspect_bytes(filepath, search_string):
    with open(filepath, 'rb') as f:
        content = f.read()
        index = content.find(search_string.encode('utf-8')) #or another encoding
        if index != -1:
            print(f"Found '{search_string}' at byte index: {index}")
            print(f"Bytes around it: {content[index-10:index+10]}")
        else:
            print(f"'{search_string}' not found.")

inspect_bytes('/content/drive/MyDrive/cleaned_output_utf8.csv', 'â€“')

'â€“' not found.
